## Web APIs

Utilizando uma API web para ler dados de um CEP

In [2]:
import requests
cep = '13083886'
url = 'https://viacep.com.br/ws/{cep}/json/'
url = url.format(cep=cep)
r = requests.get(url)
print(r.text)

{
  "cep": "13083-886",
  "logradouro": "Avenida André Tosello",
  "complemento": "",
  "bairro": "Cidade Universitária",
  "localidade": "Campinas",
  "uf": "SP",
  "unidade": "",
  "ibge": "3509502",
  "gia": "2446"
}


## HTTP

Podemos brincar um pouco com o protocolo HTTP usando o site [httpbin](http://httpbin.org/) 

### Fazendo um GET e inspecionando os cabeçalhos de resposta básicos

In [ ]:
# Fazendo um GET no site
resposta = requests.get('http://httpbin.org/')

# Verificando o código HTTP da resposta
resposta.status_code

In [ ]:
# Imprimindo os cabeçalhos da resposta
print(resposta.headers)

In [ ]:
# Qual o tipo do conteúdo da resposta?
print(resposta.headers['Content-Type'])

In [ ]:
# Qual o tamanho da resposta?
print(resposta.headers['Content-Length'])

In [ ]:
# Fazendo um GET em https://api.ipify.org/
meu_ip = requests.get('https://api.ipify.org/')

# Imprimindo o conteúdo da resposta
print(meu_ip.text)

### Negociando o tipo do conteúdo usando cabeçalhos de pedido

In [ ]:
# Pedindo por uma imagem, sem especificar nada
imagem = requests.get('http://httpbin.org/image')

# Qual o tipo da imagem retornado?
print(imagem.headers['Content-Type'])

In [ ]:
# application/json não é um tipo de imagem! O que aconteceu?
print(imagem.text)

In [ ]:
# Qual foi o código de resposta?
print(imagem.status_code)

In [ ]:
# Precisamos inserir um cabeçalho, especificando que queremos uma imagem
cabeçalho = {'accept': 'image/*'}  # Cabeçalho que pede pelo tipo PNG
imagem = requests.get('http://httpbin.org/image', headers=cabeçalho)

# Qual o tipo agora?
print(imagem.headers['Content-Type'])

In [ ]:
# Qual o código da resposta?
print(imagem.status_code)

In [ ]:
# Qual o tamanho da resposta?
print(imagem.headers['Content-Length'])

In [ ]:
# Exibindo a imagem!
from IPython.display import Image
Image(imagem.content)

### Outros códigos de resposta

In [ ]:
# Tentando acessar uma página proibida!
página_proibida = requests.get('http://httpbin.org/status/403')

print(página_proibida.status_code)

In [ ]:
# E uma página que não existe?
página_inexistente = requests.get('http://httpbin.org/blablabla')
print(página_inexistente.status_code)

In [ ]:
# Tentando acessar uma página que me redireciona para outra.
# Utilizei a flag `allow_redirects=False` para que a biblioteca `requests` não me redirecione automaticamente
# Experimente fazer o pedido alterando a flag para True!
redirecionamento = requests.get('http://httpbin.org/redirect-to?url=http://2016.pythonbrasil.org.br/',
                                allow_redirects=False)

# Qual foi o código de resposta?
print(redirecionamento.status_code)

In [ ]:
# Quais os cabeçalhos da resposta?
print(redirecionamento.headers)

In [ ]:
# Observe o cabeçalho `Location`!
print(redirecionamento.headers['Location'])

### Enviando parâmetros e dados via HTTP

In [ ]:
# Nós podemos especificar parâmetros ao fazer um pedido do tipo GET
parâmetros = {
    'data': '22/10/2012',
    'local': 'Brasil'
}

r = requests.get('http://httpbin.org/get', params=parâmetros)

# O HTTP Bin nos retorna no corpo da resposta os parâmetros que a gente enviou
print(r.text)

In [3]:
# Utilizando o verbo POST, podemos enviar dados para serem processados pela aplicação
dados = {
    'nome': 'João',
    'idade': 20
}

r = requests.post('http://httpbin.org/post', data=dados)

# Assim como no caso do GET, o HTTP Bin retorna os dados que enviamos no POST
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "idade": "20", 
    "nome": "Jo\u00e3o"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "23", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.11.1"
  }, 
  "json": null, 
  "origin": "177.8.106.72", 
  "url": "http://httpbin.org/post"
}

